In [11]:
#importing pertinent libraries
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import sqlite3, requests, re
import json

In [17]:
#extracting UFC data

#creating a list of unique fighter URLs available on website
fighter_urls = []
response = requests.get('http://www.ufcstats.com/statistics/fighters?char=a&page=all')
soup = BeautifulSoup(response.content, 'html.parser')

#targeting URLs that contain fighter data
fighters = soup.findAll('a', attrs={'href': re.compile('http:')})
for fighter in fighters:
    fighter_urls.append(fighter.get('href'))

#removing duplicate URLs
fighter_urls = list(set(fighter_urls))
fighter_urls

['http://www.ufcstats.com/fighter-details/a08ddd04eaffd81d',
 'http://www.ufcstats.com/fighter-details/04643fd03d9159cc',
 'http://www.ufcstats.com/fighter-details/44aa652b181bcf68',
 'http://www.ufcstats.com/fighter-details/15df64c02b6b0fde',
 'http://www.ufcstats.com/fighter-details/c482c8605455a213',
 'http://www.ufcstats.com/fighter-details/4b37a0bc2ab4cae1',
 'http://www.ufcstats.com/fighter-details/4665f9f909cd7214',
 'http://www.ufcstats.com/fighter-details/9199e0735b83dd32',
 'http://www.ufcstats.com/fighter-details/2b074403b7c6cdb4',
 'http://www.ufcstats.com/fighter-details/ff222167cce02919',
 'http://www.ufcstats.com/fighter-details/6eaec40f724852f4',
 'http://www.ufcstats.com/fighter-details/d156513a19acf856',
 'http://www.ufcstats.com/fighter-details/770b9d4813c25902',
 'http://www.ufcstats.com/statistics/fighters?char=a&page=3',
 'http://www.ufcstats.com/fighter-details/ba5c6eeb8ef45f5c',
 'http://www.ufcstats.com/fighter-details/956dcac65e7b34d6',
 'http://www.ufcstats.c

In [18]:
#removing invalid URLs
for url in fighter_urls:
    if 'fighter-details' not in url:
        fighter_urls.remove(url)

In [26]:
fighter_urls

['http://www.ufcstats.com/fighter-details/a08ddd04eaffd81d',
 'http://www.ufcstats.com/fighter-details/04643fd03d9159cc',
 'http://www.ufcstats.com/fighter-details/44aa652b181bcf68',
 'http://www.ufcstats.com/fighter-details/15df64c02b6b0fde',
 'http://www.ufcstats.com/fighter-details/c482c8605455a213',
 'http://www.ufcstats.com/fighter-details/4b37a0bc2ab4cae1',
 'http://www.ufcstats.com/fighter-details/4665f9f909cd7214',
 'http://www.ufcstats.com/fighter-details/9199e0735b83dd32',
 'http://www.ufcstats.com/fighter-details/2b074403b7c6cdb4',
 'http://www.ufcstats.com/fighter-details/ff222167cce02919',
 'http://www.ufcstats.com/fighter-details/6eaec40f724852f4',
 'http://www.ufcstats.com/fighter-details/d156513a19acf856',
 'http://www.ufcstats.com/fighter-details/770b9d4813c25902',
 'http://www.ufcstats.com/fighter-details/ba5c6eeb8ef45f5c',
 'http://www.ufcstats.com/fighter-details/956dcac65e7b34d6',
 'http://www.ufcstats.com/fighter-details/c11036da9162cb5f',
 'http://www.ufcstats.co

In [82]:
tester = fighter_urls[0]
response = requests.get(tester)
soup = BeautifulSoup(response.content, 'html.parser')
tester_info = soup.findAll('h2', attrs={'class': re.compile('b-content')})
tester_info

[<h2 class="b-content__title">
 <span class="b-content__title-highlight">
                 Mariya Agapova
 
             </span>
 <span class="b-content__title-record">
                 Record: 6-1-0
             </span>
 </h2>]

In [95]:
soup.findChildren('span', attrs={'class': re.compile('b-content__title-highlight')})

[<span class="b-content__title-highlight">
                 Mariya Agapova
 
             </span>]

In [91]:
for i in tester_info:
    print(i.findChildren('span', ))

[<span class="b-content__title-highlight">
                Mariya Agapova

            </span>, <span class="b-content__title-record">
                Record: 6-1-0
            </span>]


In [ ]:
b-content__title-highlight